In [1]:
#Import Libraries
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown,display
from openai import OpenAI

In [2]:
#Do this if you have an API Key
load_dotenv(override=True)


True

In [3]:
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook


In [4]:
open_ai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [5]:
open_ai

In [6]:
#First API call
message = "Hello, GPT! This is my first ever message to you! Hi!"
response = open_ai.chat.completions.create(model="llama3.2", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! It's wonderful to meet you for the first time. I'm excited to be a part of your conversational journey and help with any questions or topics you'd like to discuss. How are you doing today? Is there something specific on your mind that you'd like to talk about, or do you just want to get to know me better?


In [7]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self,url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url,headers)

        print(f"Status Code: {response.status_code}")  # Check if request succeeded
        print(f"Content Type: {response.headers.get('Content-Type')}")
        print(f"Response Size: {len(response.text)} bytes")
        #print(f"Content: {response.content}")
        soup = BeautifulSoup(response.content,'html.parser')
        #print(f"Content: {soup}")
        self.title = soup.title.string if soup.title else "No title Found"
        for irrelevant in soup.body(["script","style","img","input"]):
            irrelevant.decompose()

        self.text = soup.body.get_text(separator = "\n", strip = True)
        

In [8]:
ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Status Code: 200
Content Type: text/html; charset=UTF-8
Response Size: 93422 bytes
Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, an

In [9]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [10]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [11]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

# Example of Messages

In [12]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [13]:
# To give you a preview -- calling OpenAI with system and user messages:

response = open_ai.chat.completions.create(model="llama3.2", messages=messages)
print(response.choices[0].message.content)

*sigh* Oh, wow. I'm shocked you even need to ask me that question. It's not like it's elementary school math or anything. Fine. The answer is... (dramatic pause) ...4. Happy now? Can I go back to my busy schedule of solving actual difficult problems for real people?


# Actual Code

In [14]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ] 

In [15]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

In [20]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = open_ai.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [21]:
summarize("https://edwarddonner.com")

Status Code: 200
Content Type: text/html; charset=UTF-8
Response Size: 93422 bytes


'# Website Summary\n```markdown\n## Overview\nThe website "Home - Edward Donner" is the homepage of a personal website. It appears to be the author\'s blog or profile page.\n\n### Author Information\nEd Donner is the author, who enjoys writing code and experimenting with LLMs (Large Language Models). He mentions being part of two AI startups, Nebula.io and untapt (acquired in 2021).\n\n### Recent News/Announcements\n```markdown\n* **January 23, 2025**: [LLM Workshop – Hands-on with Agents](link to resource)\n* **December 21, 2024**: Welcome to the LLM Workshop!\n* **November 13, 2024**: Welcome, SuperDataScientists!\n* **October 16, 2024**: Mastering AI and LLM Engineering \n```\n\n### Notable Links\n```markdown\n**Projects**\n* [Connect Four](link)\n* Outsmart (description: "an arena that pits LLMs against each other")\n```\nNote: The "Connect" section appears to be a navigation menu, which has been ignored for this summary.\n```plaintext\n# Removed from summary \n Navigation\nHome\nC

In [22]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [23]:
display_summary("https://edwarddonner.com")

Status Code: 200
Content Type: text/html; charset=UTF-8
Response Size: 93422 bytes


# Summary of Edward Donner's Website

### About the Author

Ed Donner is a co-founder and CTO of Nebula.io, an AI platform that helps people discover their potential. He has experience in DJing, amateur electronic music production, and reading Hacker News.

### Recent Announcements (News)

*   **LLM Workshop – Hands-on with Agents** - January 23, 2025
    * A resource page for those interested in hands-on training for LLM work, using agents.
*   **Welcome, SuperDataScientists!** - November 13, 2024
    * An open invitation to the community to get involved.
*   **Mastering AI and LLM Engineering** - October 16, 2024
    * Resources for those interested in learning more about this specialized field.

### Featured Projects

*   Connect Four: A game where players take turns dropping pieces with the goal of having four pieces in a row, both horizontally, vertically, or diagonally.
*   Outsmart: An arena for exploring diplomacy and deviousness between LLMs.

In [24]:
display_summary("https://cnn.com")

Status Code: 200
Content Type: text/html; charset=utf-8
Response Size: 3096417 bytes


Here are the top news stories from around the world, based on the provided text:

1. **Deadly earthquake strikes Myanmar, Thailand, and China**: A magnitude 7.5 earthquake struck the region, causing significant damage and loss of life.
2. **Anti-migrant policies leave Afghan pilots fearing for their lives**: The US government's policy on restricting air travel to Afghan nationals has made it difficult for pilots to return to Afghanistan amid the ongoing conflict.
3. **China is practicing 'dogfighting' with satellites as it ramps up space capabilities**: The US Space Force has warned that China is developing its own anti-satellite missile technology, which could pose a significant threat to global satellite networks.
4. **Apple needs a new reason to get you to buy a new iPhone**: Experts say Apple may be focusing on AI-powered features and sustainable design as the next major selling point for the iPhone.
5. **Trump admin invokes state secrets privilege over deportation flight information**: A federal judge has temporarily blocked the Trump administration's decision to withhold information about flights that carried migrants detained at the US-Mexico border.
6. **'Get us out of here': Anti-migrant policies leave Afghan pilots fearing for their lives**: (same as above)

Other notable stories include:

* The 2025 reveal of Asia's 50 best restaurants
* A long-lost Klimt painting of an African prince goes on show
* Kathy Bates says work feels 'magnificent' after losing 100 lbs.
* CNN Fast: Exploring new developments in AI-powered medical care

Please note that these stories are condensed from the original content provided and may not be a comprehensive list of all news stories.

In [25]:
display_summary("https://anthropic.com")

Status Code: 403
Content Type: text/html; charset=UTF-8
Response Size: 7788 bytes


# Summary of Website "Just a Moment..."

## Main Content:

This website appears to be a brief apology or warning message from the administrators.

### Key Text:

* Enable JavaScript and cookies to continue
* No additional content provided.

It seems that the website is empty except for this note, so no further information can be confirmed.